# **SQL PRORAMMING**

**Overview**

In this part of the assessment, you will act as a Data analyst to answer a research question on the US elections. 

First, will be required to write a Data Report on the process that will undertake while working on the given research problem. Using the CRISP-DM Methodology,  you will document the various stages of the data science lifecycle for the given research problem while also providing your recommendation.

You have been provided with a detailed description of what you will be expected to do below.

Do remember that complex tasks that may seem hard at first can often be broken down into a sequence of simple tasks, and there are workarounds to do what first seems “impossible” with a succession of known operations.

**Problem Description** 

You have been asked to help a candidate to become US president. The winner of the election will be the candidate winning the most grand electors. 

Grand electors are attributed at the state level: in each of the 51 states, there is a given number of grand electors to win (roughly, but not exactly, proportional to the size of the state) and the presidential candidate receiving the most local votes wins ALL the Grand Electors in that state.

Because the number of grand electors is not exactly proportional to the population, some states can be prioritized to increase the return on investment of the campaign. We assume here there are only 2 candidates, and no history (no trend of certain states to vote for a particular candidate or party). Hence, each vote is equally "expensive" to get, but some states grant more grand elector per capita.

You are provided with 2 tables: one giving the number of Grand Electors per state, the other the population per state.

You are asked to identify the states that should be prioritized to win the election, with a smart but simple algorithm (brute force computation of all possibilities to find the absolute optimum is not accepted, as it would be to computationally expensive). It is ok not to find the overall optimum, we just want a strategy that makes sense.

(This is of course a very simplistic view of reality, but it is a nice starting point to play with data and analyze possibilities).

First take a few minutes to think about what you need to do. Steps will be suggested hereafter, but a big part of the data scientist's job is to identify the flow of operations when being asked a practical question, so it is important you start exercising on that in addition to programming.

Here is what we are suggesting to do: we will rank states by decreasing number of grand electors per capita. The first states in the list will be the most valuable (you get a large number of grand electors by convincing a small number of people to vote for you). We will target all the states at the top of the list until the cumulative sum (also called running total) of grand electors won is larger than half the total number of Grand Electors in the country.

To do that, we need (you are allowed to create as many intermediary tables as you want, to keep queries short):

1. To join the 2 tables:
   * You notice States are not capitalized the same way in both tables (one is  in uppercase letters, the other not), so you will first need to convert all to uppercase, for instance.
   * Now you can join the tables on the state key.

2. Your boss wants you to change the name of the "District of Columbia" state to its short version "DC". Please do that.
3. To compute the ratio between the number of grand electors and the population. Please create a new column with that ratio.
4. To order the states by decreasing ratio of Grand Electors per capita. That will make our priority list.
5. To compute the running total of Grand Electors in that sorted list.
   * Hint: you can get inspiration from here to compute a running total from here:  https://stackoverflow.com/questions/21382766/cumulative-summing-values-in-sqlite
6. Independently, to compute the half of the total of Grand Electors overall (in the whole country):
  * This is the threshold we need to reach for winning the presidential election.
7. To filter our sorted list of states in order to keep only the (top) ones enabling us to reach the computed threshold. (the other states can be ignored). That is our target list.
   * Hint: You can do that in 2 steps:
       * Select all the states for which the running total is below or equal to the threshold.
       * Add the first state for which the running total is larger than the threshold.

Can you draw some conclusions from the result? Is it in line with your expectations? How many states do you end up with in the target list? Is it a small or a large number? Do you think it would be a good recommendation to target those states?    

**Datasets**

Below are two datasets that can be considered for research:

 1. Grand electors by State. [link text](https://drive.google.com/a/moringaschool.com/file/d/1AuPlNXKjmEdRTLGqKbP-OP1XZ5UQPIPw/view?usp=sharing)
 2. Population by State. [link text](https://drive.google.com/a/moringaschool.com/file/d/1VKt_hF2pRqPxcNb1DKotkVXWNd2HX_KL/view?usp=sharing)

# **1.0 CONNECTING TO THE DATABASE**

In [568]:
# loading the sql extension into the environment
%load_ext sql
# connect into the sqlite database memory
%sql sqlite://

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @None'

In [569]:
# Import pands library into the environment
import pandas as pd

# **2.0 Loading the datasets**

In [570]:
# loading the Grand electors by State dataset into the environment
with open('/content/GrandElectors_by_state.csv','r') as f:
  grand_electors = pd.read_csv(f, index_col = 0, encoding = 'utf-8')
%sql DROP TABLE if EXISTS grand_electors;
%sql PERSIST grand_electors;
%sql SELECT * FROM grand_electors LIMIT 5;

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


State,GrandElectors
Alabama,9
Alaska,3
Arizona,11
Arkansas,6
California,55


In [571]:
# loading the Population by State dataset into the environment
with open('/content/Population_by_state.csv','r') as f:
  population = pd.read_csv(f, index_col = 0, encoding = 'utf-8')
%sql DROP TABLE if EXISTS population;
%sql PERSIST population;
%sql SELECT * FROM population LIMIT 5;

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


State,Population
CALIFORNIA,39144818
TEXAS,27469114
FLORIDA,20271272
NEW YORK,19795791
ILLINOIS,12859995


# **3.0 Cleaning the Data**

In [572]:
# Convert the data in state column of the grand_electors dataset into 
# upper case to march that one in the population dataset
%%sql
UPDATE grand_electors
SET State = UPPER(State);
SELECT * FROM grand_electors LIMIT 5;

 * sqlite://
51 rows affected.
Done.


State,GrandElectors
ALABAMA,9
ALASKA,3
ARIZONA,11
ARKANSAS,6
CALIFORNIA,55


**Changing the name of the "District of Columbia" state to its short version "DC". Please do that.**

In [573]:
# For the population dataset
%%sql
UPDATE population
SET State = 'DC'
WHERE State = 'DISTRICT OF COLUMBIA';
SELECT * FROM population WHERE State = 'DC';

 * sqlite://
1 rows affected.
Done.


State,Population
DC,672228


In [574]:
# For the grand_electors dataset
%%sql
UPDATE grand_electors
SET State = 'DC'
WHERE State = 'DISTRICT OF COLUMBIA';
SELECT * FROM grand_electors WHERE State = 'DC';

 * sqlite://
1 rows affected.
Done.


State,GrandElectors
DC,3


# **4.0 Joining the two tables**

First check which states don't exist in both the datasets

In [575]:
# Which state exist only once in both datasets
%%sql
SELECT State FROM grand_electors
WHERE NOT EXISTS (SELECT * FROM population WHERE grand_electors.State = population.State);

 * sqlite://
Done.


State


The preview how the joined data will look like

In [576]:
%%sql
SELECT grand_electors.State,GrandElectors, population.Population FROM grand_electors
INNER JOIN population ON  grand_electors.State = population.State LIMIT 5;

 * sqlite://
Done.


State,GrandElectors,Population
ALABAMA,9,4858979
ALASKA,3,738432
ARIZONA,11,6828065
ARKANSAS,6,2978204
CALIFORNIA,55,39144818


# **4.1 Creating a new table**

In [577]:
# using the function 'CREATE TABLE' to come up with a new table
# known as merged_data
# 
%%sql
CREATE TABLE merged_data (
    State char,
    GrandElectors int,
    Population int
);
SELECT * FROM merged_data;

 * sqlite://
Done.
Done.


State,GrandElectors,Population


Copy the contents from joinig data quey into the new table

In [578]:
%%sql
INSERT INTO merged_data SELECT grand_electors.State,GrandElectors, population.Population FROM grand_electors
INNER JOIN population ON  grand_electors.State = population.State;

 * sqlite://
51 rows affected.


[]

In [579]:
# previewing the data in the new table
%%sql
SELECT * FROM merged_data LIMIT 5;

 * sqlite://
Done.


State,GrandElectors,Population
ALABAMA,9,4858979
ALASKA,3,738432
ARIZONA,11,6828065
ARKANSAS,6,2978204
CALIFORNIA,55,39144818


# **5.0 Working with the new table** (merged_data)

Creating a new column known as ratio.



In [580]:
# A new column
# to compute the ratio between the number of grand electors and the population.
%%sql
ALTER TABLE merged_data ADD "ratio(x10^6)" double;
SELECT * FROM merged_data LIMIT 5;

 * sqlite://
Done.
Done.


State,GrandElectors,Population,ratio(x10^6)
ALABAMA,9,4858979,None
ALASKA,3,738432,None
ARIZONA,11,6828065,None
ARKANSAS,6,2978204,None
CALIFORNIA,55,39144818,None


Updating data into the new column

In [581]:
%%sql
UPDATE merged_data
SET "ratio(x10^6)" = (GrandElectors*1000000)/(Population);
SELECT * FROM merged_data LIMIT 5;

 * sqlite://
51 rows affected.
Done.


State,GrandElectors,Population,ratio(x10^6)
ALABAMA,9,4858979,1.0
ALASKA,3,738432,4.0
ARIZONA,11,6828065,1.0
ARKANSAS,6,2978204,2.0
CALIFORNIA,55,39144818,1.0


In [582]:
# %%sql
# DROP TABLE merged_data;
# DROP TABLE merged_data_sorted;
# DROP TABLE final;

# **5.1 New sorted merged table**

Ordering the states by decreasing ratio of Grand Electors per capital. This will make our priority list.

In [583]:
%%sql
SELECT * FROM merged_data
ORDER BY "ratio(x10^6)" DESC;

 * sqlite://
Done.


State,GrandElectors,Population,ratio(x10^6)
WYOMING,3,586107,5.0
ALASKA,3,738432,4.0
DC,3,672228,4.0
VERMONT,3,626042,4.0
DELAWARE,3,945934,3.0
MAINE,4,1329328,3.0
NEW HAMPSHIRE,4,1330608,3.0
NORTH DAKOTA,3,756927,3.0
RHODE ISLAND,4,1056298,3.0
SOUTH DAKOTA,3,858469,3.0


Creating the new table

In [584]:
%%sql
CREATE TABLE merged_data_sorted(
    State char,
    GrandElectors int,
    Population int
);
ALTER TABLE merged_data_sorted ADD "ratio(x10^6)" double;
SELECT * FROM merged_data_sorted LIMIT 5;

 * sqlite://
Done.
Done.
Done.


State,GrandElectors,Population,ratio(x10^6)


Updating the new table with the sorted data


In [585]:
%%sql
INSERT INTO merged_data_sorted SELECT * FROM merged_data
ORDER BY "ratio(x10^6)" DESC;

 * sqlite://
51 rows affected.


[]

In [586]:
# previewing the merged_data_sorted table
%%sql
SELECT * FROM merged_data_sorted LIMIT 5;

 * sqlite://
Done.


State,GrandElectors,Population,ratio(x10^6)
WYOMING,3,586107,5.0
ALASKA,3,738432,4.0
DC,3,672228,4.0
VERMONT,3,626042,4.0
DELAWARE,3,945934,3.0


Finding the cumulative sum of the grand electors

In [600]:
%%sql
select t1.State, t1.GrandElectors, t1.Population, t1."ratio(x10^6)",
SUM(t2.GrandElectors) as cummulative_sum
from merged_data_sorted t1
inner join merged_data_sorted t2 on t1."ratio(x10^6)" == t2."ratio(x10^6)"
group by t1.State, t1.GrandElectors, t1.Population, t1."ratio(x10^6)"
order by t1."ratio(x10^6)" DESC;

 * sqlite://
Done.


State,GrandElectors,Population,ratio(x10^6),cummulative_sum
WYOMING,3,586107,5.0,3
ALASKA,3,738432,4.0,9
DC,3,672228,4.0,9
VERMONT,3,626042,4.0,9
DELAWARE,3,945934,3.0,21
MAINE,4,1329328,3.0,21
NEW HAMPSHIRE,4,1330608,3.0,21
NORTH DAKOTA,3,756927,3.0,21
RHODE ISLAND,4,1056298,3.0,21
SOUTH DAKOTA,3,858469,3.0,21


# **5.2 Final working on table**

In [588]:
# creating a new table called final to work on the new data with 
# cumulative sum in it
%%sql
CREATE TABLE final(
    State char,
    GrandElectors int,
    Population int
);
ALTER TABLE final ADD "ratio(x10^6)" double;
ALTER TABLE final ADD cummulative_sum double;
SELECT * FROM final;

 * sqlite://
Done.
Done.
Done.
Done.


State,GrandElectors,Population,ratio(x10^6),cummulative_sum


In [589]:
# updating the new table
%%sql
INSERT INTO final 
  select t1.State, t1.GrandElectors, t1.Population, t1."ratio(x10^6)",
  SUM(t2.GrandElectors) as cummulative_sum
  from merged_data_sorted t1
  inner join merged_data_sorted t2 on t1."ratio(x10^6)" == t2."ratio(x10^6)"
  group by t1.State, t1.GrandElectors, t1.Population, t1."ratio(x10^6)"
  order by t1."ratio(x10^6)" DESC;

 * sqlite://
51 rows affected.


[]

In [590]:
#  preview the new table
%%sql 
SELECT * FROM final;

 * sqlite://
Done.


State,GrandElectors,Population,ratio(x10^6),cummulative_sum
WYOMING,3,586107,5.0,3.0
ALASKA,3,738432,4.0,9.0
DC,3,672228,4.0,9.0
VERMONT,3,626042,4.0,9.0
DELAWARE,3,945934,3.0,21.0
MAINE,4,1329328,3.0,21.0
NEW HAMPSHIRE,4,1330608,3.0,21.0
NORTH DAKOTA,3,756927,3.0,21.0
RHODE ISLAND,4,1056298,3.0,21.0
SOUTH DAKOTA,3,858469,3.0,21.0


**Computing the half total of GrandElectors overall(also known as threshold)**

In [596]:
%%sql
SELECT SUM(GrandElectors)*0.5 AS "Half sum of the total Grand Electors" FROM FINAL

 * sqlite://
Done.


Half sum of the total Grand Electors
269.0


**Selecting all the states for which the running total is below or equal to the threshold.**

In [602]:
%%sql
SELECT * FROM final
WHERE cummulative_sum <= (SELECT SUM(GrandElectors)*0.5 FROM FINAL);

 * sqlite://
Done.


State,GrandElectors,Population,ratio(x10^6),cummulative_sum
WYOMING,3,586107,5.0,3.0
ALASKA,3,738432,4.0,9.0
DC,3,672228,4.0,9.0
VERMONT,3,626042,4.0,9.0
DELAWARE,3,945934,3.0,21.0
MAINE,4,1329328,3.0,21.0
NEW HAMPSHIRE,4,1330608,3.0,21.0
NORTH DAKOTA,3,756927,3.0,21.0
RHODE ISLAND,4,1056298,3.0,21.0
SOUTH DAKOTA,3,858469,3.0,21.0


In [605]:
# Determing the number of states to priotirise the elections in
%%sql
SELECT COUNT(State) FROM final
WHERE cummulative_sum <= (SELECT SUM(GrandElectors)*0.5 FROM FINAL);

 * sqlite://
Done.


COUNT(State)
21
